# Realiza o merge de um conjunto de arquivos texto de uma pasta para um único arquivo, depois aplica as técnicas de pré-processamento e filtros

In [41]:
import numpy as np
import re
import nltk
from collections import Counter
import os
from pathlib import Path
import datetime
from string import ascii_lowercase
import pandas as pd

In [4]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [6]:
# Iterando os arquivos da pasta atual
import os
from pathlib import Path
import datetime

extensao = "*.txt"
pastaentrada =  os.path.join("..", "..", "_Corpora", "Glossarios_siglarios")
pastaSaida = "ANP"
arquivoSaida = os.path.join(pastaSaida, 'anp_glossarios.txt')
arquivoCorpusFinal = os.path.join(pastaSaida, 'anp_glossarios_final.txt')
print("Pasta de entrada: ", pastaentrada)

tokens = 0
sentencas_totais = 0

# cria a pasta de saida, se nao existir
if not os.path.exists(pastaSaida):
    os.makedirs(pastaSaida)

Pasta de entrada:  ..\..\_Corpora\Glossarios_siglarios


In [7]:
momentoInicial = datetime.datetime.now()

pastaArquivosCorpus = Path(pastaentrada).glob(extensao)

for path in pastaArquivosCorpus:
    path_arquivo = str(path) # because path is object not string
    print("Lendo o arquivo ", path_arquivo)
    
    texto = ''
    with open(path_arquivo, 'r', encoding="UTF-8") as f:
        texto = f.read()
    
    with open(arquivoSaida, 'a', encoding='utf8') as fp:
        fp.write('%s' % texto)
        
momentoFinal = datetime.datetime.now()
print("Tempo total decorrido: ", momentoFinal - momentoInicial)

Lendo o arquivo  ..\..\_Corpora\Glossarios_siglarios\glossario Petrobras.txt
Lendo o arquivo  ..\..\_Corpora\Glossarios_siglarios\glossarioANP.txt
Lendo o arquivo  ..\..\_Corpora\Glossarios_siglarios\siglario.txt
Tempo total decorrido:  0:00:00.047000


In [8]:
def leTextoDeArquivo(arquivoEntrada):
    with open(arquivoEntrada, 'r', encoding="UTF-8") as f:
        texto = f.read()
        
    #print(texto[:200])
    return texto

In [9]:
def gravaArquivo(fileName, texto):
    with open(fileName, 'w', encoding='utf8') as fp:
        fp.write('%s\n' % texto)

In [10]:
def imprime_info_corpus(texto, plot=False):
    word_counts = Counter()
    tokens_totais = 0
    # Tokenize and remove short and malformed sentences.
    for sent in texto.split():
        #print(sent)
        tokens_totais += sent.count(' ') + 1
        word_counts[sent] += 1
        
    print('Total de tokens analisados: ', "{:,}".format(tokens_totais))
    print('Tamanho do Vocabulário: ', "{:,}".format(len(word_counts)))
    print("\n")
    print(word_counts.most_common(300))
    
    most_common = []
    for common_word in word_counts.most_common(1000):
        most_common.append(common_word[1])

    if plot:
        hist, edges = np.histogram(most_common, density=True, bins=100, normed=True)

        p = figure(tools="pan,wheel_zoom,reset,save",
                   toolbar_location="above",
                   title="Top-1000 words distribution")
        p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
        show(p)  

In [42]:
texto = leTextoDeArquivo(arquivoSaida)

In [43]:
imprime_info_corpus(texto)

Total de tokens analisados:  47,470
Tamanho do Vocabulário:  12,215


[('de', 4660), ('e', 1203), ('a', 754), ('do', 692), ('ou', 602), ('em', 522), ('o', 470), ('(Fonte:', 468), ('que', 465), ('nº', 444), ('da', 443), ('para', 366), ('ANP', 317), ('gás', 293), ('petróleo', 286), ('com', 274), ('Resolução', 265), ('no', 250), ('por', 236), ('na', 206), ('um', 199), ('uma', 166), ('como', 151), ('natural', 148), ('dos', 140), ('é', 138), ('Gás', 138), ('à', 136), ('as', 135), ('ao', 131), ('produção', 131), ('and', 126), ('das', 122), ('of', 119), ('os', 116), ('pela', 112), ('pelo', 104), ('ser', 100), ('não', 99), ('Portaria', 97), ('Ver', 96), ('entre', 88), ('-', 82), ('1,', 82), ('Petróleo', 81), ('se', 78), ('volume', 78), ('Conjunta', 77), ('ANP/Inmetro', 75), ('mais', 72), ('Natural', 70), ('Lei', 70), ('Volume', 65), ('óleo', 63), ('Dicionário', 63), ('poço', 62), ('nas', 62), ('10/6/2013).', 62), ('Poço', 60), ('petróleo,', 58), ('Produção', 56), ('são', 55), ('transporte', 55

## Eliminando caracteres acentuados

Observou-se a ocorrência de muitas palavras com divergência de acentuação (a mesma palavra ocorria escrita com e sem acentos). O agrupamento ajudou a identificar corretamente a grafia de uma mesma palavra.

In [44]:
texto = str(texto.lower())
import unicodedata
nfkd_form = unicodedata.normalize('NFKD', texto)
texto= u"".join([c for c in nfkd_form if not unicodedata.combining(c)])
#print("\nDepois: ", texto[15000:17000])

In [45]:
imprime_info_corpus(texto)

Total de tokens analisados:  47,470
Tamanho do Vocabulário:  10,914


[('de', 4662), ('e', 1386), ('a', 939), ('no', 709), ('do', 693), ('ou', 603), ('em', 535), ('o', 523), ('gas', 470), ('(fonte:', 468), ('que', 465), ('da', 451), ('para', 374), ('petroleo', 367), ('anp', 317), ('com', 277), ('resolucao', 265), ('por', 246), ('natural', 218), ('na', 208), ('um', 203), ('producao', 187), ('as', 172), ('uma', 170), ('como', 151), ('volume', 143), ('dos', 140), ('ao', 131), ('and', 126), ('os', 123), ('das', 122), ('poco', 122), ('of', 122), ('pela', 112), ('nao', 111), ('oleo', 111), ('pelo', 104), ('ser', 100), ('portaria', 99), ('ver', 98), ('entre', 89), ('sistema', 83), ('-', 82), ('1,', 82), ('se', 80), ('conjunta', 78), ('petroleo,', 77), ('anp/inmetro', 77), ('transporte', 76), ('mais', 74), ('lei', 71), ('sao', 70), ('combustivel', 70), ('unidade', 70), ('area', 66), ('condicoes', 66), ('pressao', 64), ('dicionario', 64), ('nas', 62), ('conjunto', 62), ('10/6/2013).', 62), ('da

In [46]:
#texto_sem_pontuacao = texto_sem_acentos.replace('["#%\'()*+,-/:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]', ' ')

# primeiro, adicionando espaços aos caracteres de pontuacao
import re
texto = re.sub('([.,!?()/])', r' \1 ', texto)
texto = re.sub('\s{2,}', ' ', texto)

In [47]:
#retirando os caracteres de pontuação e numeros (mantendo apenas os caracteres de ponto, final de sentença)
pontuacao = '["#%\'()*+,-/:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\•]'
texto = ''.join([c for c in texto if c not in pontuacao])

In [48]:
imprime_info_corpus(texto)

Total de tokens analisados:  50,599
Tamanho do Vocabulário:  7,534


[('de', 4664), ('.', 4179), ('e', 1428), ('a', 961), ('no', 709), ('do', 693), ('ou', 648), ('o', 563), ('em', 542), ('gas', 541), ('petroleo', 504), ('que', 491), ('fonte', 484), ('da', 451), ('anp', 442), ('para', 377), ('natural', 330), ('producao', 283), ('com', 278), ('resolucao', 272), ('por', 248), ('na', 209), ('um', 206), ('poco', 199), ('as', 172), ('uma', 170), ('como', 156), ('volume', 155), ('oleo', 152), ('dos', 140), ('ao', 131), ('and', 126), ('os', 125), ('of', 124), ('das', 122), ('pressao', 120), ('nao', 119), ('pela', 112), ('sistema', 106), ('ver', 105), ('pelo', 104), ('ser', 101), ('transporte', 101), ('portaria', 101), ('combustivel', 100), ('medicao', 93), ('entre', 89), ('unidade', 89), ('inmetro', 89), ('agua', 85), ('se', 81), ('area', 80), ('conjunta', 78), ('lei', 77), ('mais', 74), ('combustiveis', 74), ('derivados', 74), ('port', 74), ('nacional', 71), ('sao', 70), ('dados', 70), ('cond

# retirar caracteres unicos e stopwords

### Eliminação de stopwords e palavras incorretas do vocabulário

Em estudos anteriores, observou-se a presença preponderante de um conjunto de palavras incorretas no vocabulário mais comum, potencialmente resultantes de incorreções no processo de extração de PDF para texto.
Neste trecho, define-se uma lista contendo todas essas palavras incorretas, palavras de um único caractere, e stopwords dos idiomas português, inglês e espanhol. Observou-se que o histograma de distribuição do vocabulário comportou-se melhor após este processamento.

In [49]:
from nltk.corpus import stopwords # Import the stop word list
from string import ascii_lowercase
#nltk.download('stopwords')

# Mapeando stopwords com NLTK
stopwordsPortugues = stopwords.words("portuguese")
stopwordsIngles = stopwords.words("english")
stopwordsEspanhol = stopwords.words("spanish")
caracteres_unicos = [c for c in ascii_lowercase ]

# em estudo anterior, obtivemos o conjunto das palavras mais comuns, onde se observam algumas palavras incorretas
# resultantes do processo de extração do arquivo PDF. Estas palavras serão incluídas no vocabulário a ser ignorado
# no corpus final.
# Objetiva-se incrementar esta lista conforme novas palavras são observadas, à medida em que novos arquivos são
# acrescentados ao corpus
palavrasIncorretasComuns = ['&', 'ch', 'et', 'at', 'ii','iii', '\x05', '•', '–', '°']

# mais rapido buscar set do que list
vocabulario_palavras_ignoradas = set(stopwordsPortugues + 
                                     stopwordsIngles + 
                                     stopwordsEspanhol + 
                                     caracteres_unicos + 
                                     palavrasIncorretasComuns)

print("Palavras a serem ignoradas do vocabulario", vocabulario_palavras_ignoradas)

Palavras a serem ignoradas do vocabulario {'tengan', 'estuviste', 'tendréis', 'estando', 'habíamos', 'seu', 'tuas', 'hayas', 'sem', 'what', 'él', 'terei', 'você', 'ours', 'haya', 'estabais', 'teníais', "weren't", 'about', 'hubieran', 'minhas', 'del', 'tinham', 'ela', 'tuvisteis', 'haja', 'itself', 'ya', 'isn', 'houvéramos', 'seremos', 'hubieses', 'houvéssemos', 'tuvieseis', 'estemos', 'p', 'or', 'pelas', 'teníamos', 'nuestras', 'hayáis', 'uma', 'seriam', "hasn't", 'estuviese', 'foram', "didn't", 'esse', 'these', 'most', 'were', 'fuera', 'not', 'tengáis', "don't", 'fueras', 'h', 'uno', 'a', 'sentid', 'houverei', 'depois', 'tendrías', 'todos', 'serán', "aren't", 'otras', 'el', 'ao', 'he', 'hay', 'which', 'hasta', 'been', 'all', 'um', 'tanto', 'mía', 'estivesse', 'will', 'mustn', 'estivermos', 'tenho', 'through', 'estava', 'tivemos', 'doing', 'lo', 'habido', 'teus', 'forem', 'estará', 'fora', 'have', 'fôramos', 'by', 'algunos', 'ourselves', 'estivemos', 'estiveram', 'l', 'why', 'sea', 'ne

In [50]:
sentenca_sem_stopwords = [w for w in texto.split() if not w in vocabulario_palavras_ignoradas]
texto = " ".join( sentenca_sem_stopwords ) 

In [51]:
imprime_info_corpus(texto)

Total de tokens analisados:  34,935
Tamanho do Vocabulário:  7,366


[('.', 4179), ('gas', 541), ('petroleo', 504), ('fonte', 484), ('anp', 442), ('natural', 330), ('producao', 283), ('resolucao', 272), ('volume', 155), ('oleo', 152), ('pressao', 120), ('nao', 119), ('sistema', 106), ('ver', 105), ('ser', 101), ('transporte', 101), ('portaria', 101), ('combustivel', 100), ('medicao', 93), ('unidade', 89), ('inmetro', 89), ('agua', 85), ('area', 80), ('conjunta', 78), ('lei', 77), ('combustiveis', 74), ('derivados', 74), ('port', 74), ('nacional', 71), ('sao', 70), ('dados', 70), ('condicoes', 68), ('conjunto', 68), ('exploracao', 68), ('energia', 66), ('perfuracao', 65), ('dicionario', 65), ('pressure', 63), ('processo', 62), ('oil', 61), ('temperatura', 60), ('campo', 58), ('brasil', 57), ('desenvolvimento', 57), ('controle', 55), ('produtos', 53), ('portuguesa', 53), ('liquido', 52), ('lingua', 51), ('utilizado', 50), ('reservatorio', 50), ('seguranca', 50), ('base', 49), ('ponto', 4

## Existe a preponderância de muitas palavras raras no vocabulário, dominando a distribuição

Eliminando a ocorrência de palavras raras, que ocorram com menor frequência, reduzindo assim o vocabulário apenas às palaras mais relevantes. Normalmente estas palavras se referem a erros de grafia ou da extração dos PDFs para texto.

In [52]:
word_counts = Counter(texto.split())

In [53]:
print(texto[:1000])

abandono area . ato retirar forma ordenada todas pessoas area interna afetada emergencia . . processo constituido abandono pocos desativacao instalacoes area concessao . . abandono campo processo compreende abandono pocos desativacao alienacao reversao todas instalacoes producao . . abandono processo resultado processo tamponado evitar migracao fluidos varias formacoes penetradas furo . ocorre considerado subcomercial seco . . abandono permanente abandono ocorre nao interesse retorno continuidade operacoes . . abandono temporario abandono interesse retorno continuidade operacoes . . abissal relativo profundidades oceanicas alem limites plataforma continental geral . metros . comumente referido ambiente fundo . . abnt associacao brasileira normas tecnicas . instituto nacional padronizacao . correspondente eua american national standards institute ansi . . acidificacao aplicacao acido paredes pocos gas oleo remover qualquer material obstrua entrada fluido . utilizado formacoes carbonatic

In [54]:
#threshold_palavras_raras = 2 # palavras que ocorram 2 ou menos vezes serão eliminadas

#sentenca_sem_palavras_raras = [w for w in texto.split() if word_counts[w] > threshold_palavras_raras]
#texto = " ".join( sentenca_sem_palavras_raras ) 

In [55]:
#imprime_info_corpus(texto)

## Unificando as quebras de linhas, refazendo as sentenças.
Obervou-se a ocorrencia de muitas quebras de linhas desnecessárias, produto de erros na extração do PDF. As quebras de linha serão todas eliminadas, e posteriormente refeitas considerando o caractere de ponto como final de sentença e quebra de linha.

In [56]:
texto = texto.replace('\n', ' ').replace('\r', '').replace('?','.').replace('!','.')

In [57]:
lista_com_quebra_de_linha = texto.split('.')
texto = "\n".join( lista_com_quebra_de_linha ) 

In [58]:
imprime_info_corpus(texto)

Total de tokens analisados:  30,756
Tamanho do Vocabulário:  7,365


[('gas', 541), ('petroleo', 504), ('fonte', 484), ('anp', 442), ('natural', 330), ('producao', 283), ('resolucao', 272), ('volume', 155), ('oleo', 152), ('pressao', 120), ('nao', 119), ('sistema', 106), ('ver', 105), ('ser', 101), ('transporte', 101), ('portaria', 101), ('combustivel', 100), ('medicao', 93), ('unidade', 89), ('inmetro', 89), ('agua', 85), ('area', 80), ('conjunta', 78), ('lei', 77), ('combustiveis', 74), ('derivados', 74), ('port', 74), ('nacional', 71), ('sao', 70), ('dados', 70), ('condicoes', 68), ('conjunto', 68), ('exploracao', 68), ('energia', 66), ('perfuracao', 65), ('dicionario', 65), ('pressure', 63), ('processo', 62), ('oil', 61), ('temperatura', 60), ('campo', 58), ('brasil', 57), ('desenvolvimento', 57), ('controle', 55), ('produtos', 53), ('portuguesa', 53), ('liquido', 52), ('lingua', 51), ('utilizado', 50), ('reservatorio', 50), ('seguranca', 50), ('base', 49), ('ponto', 49), ('operaca

### Eliminando sentenças com menos de 3 palavras, inclusive vazias

In [62]:
lista_com_quebra_de_linha = texto.split('\n')
texto = '\n'.join([sentenca for sentenca in lista_com_quebra_de_linha if len(sentenca.split(' ')) > 3])

In [63]:
imprime_info_corpus(texto)

Total de tokens analisados:  30,680
Tamanho do Vocabulário:  7,334


[('gas', 541), ('petroleo', 504), ('fonte', 484), ('anp', 441), ('natural', 330), ('producao', 283), ('resolucao', 272), ('volume', 155), ('oleo', 152), ('pressao', 120), ('nao', 119), ('sistema', 106), ('ver', 105), ('ser', 101), ('transporte', 101), ('portaria', 101), ('combustivel', 100), ('medicao', 93), ('unidade', 89), ('inmetro', 89), ('agua', 85), ('area', 79), ('conjunta', 78), ('combustiveis', 74), ('derivados', 74), ('port', 74), ('nacional', 71), ('sao', 70), ('dados', 70), ('condicoes', 68), ('conjunto', 68), ('exploracao', 68), ('lei', 68), ('energia', 66), ('perfuracao', 65), ('dicionario', 65), ('pressure', 63), ('processo', 62), ('oil', 61), ('temperatura', 60), ('campo', 58), ('brasil', 57), ('desenvolvimento', 57), ('controle', 55), ('produtos', 53), ('portuguesa', 53), ('liquido', 52), ('lingua', 51), ('utilizado', 50), ('reservatorio', 50), ('seguranca', 50), ('base', 49), ('ponto', 49), ('operaca

In [64]:
#Checkpoint

gravaArquivo(arquivoCorpusFinal, texto)